In [1]:
import requests
import json

# Airtable credentials
AIRTABLE_BASE_ID = 'app7mcJMf9O8GQRal'
AIRTABLE_API_KEY = ''
TABLE_NAME = 'tiktok_post'
VIEW_NAME = '0_no_download'
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{TABLE_NAME}"

# TikTok API credentials
TIKTOK_API_KEY = ""
TIKTOK_API_HOST = "tiktok-api23.p.rapidapi.com"
TIKTOK_API_URL = "https://tiktok-api23.p.rapidapi.com/api/download/video"

# Headers for Airtable requests
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}"
}

# Get all records from the Airtable view
all_records = []
params = {
    "view": VIEW_NAME
}

while True:
    response = requests.get(AIRTABLE_URL, headers=airtable_headers, params=params)
    response.raise_for_status()
    response_data = response.json()
    records = response_data.get("records", [])
    all_records.extend(records)
    
    # Kiểm tra xem còn trang tiếp theo không
    offset = response_data.get("offset")
    if not offset:
        break
    params["offset"] = offset

# Thay đổi vòng lặp để sử dụng all_records
for record in all_records:
    record_id = record['id']
    fields = record['fields']
    author_uniqueid = fields.get('author_uniqueid')
    video_id = fields.get('video_id')

    # Check if required fields are present
    if not author_uniqueid or not video_id:
        print(f"Skipping record {record_id} due to missing fields.")
        continue

    # Make TikTok API request
    tiktok_url = f"https://www.tiktok.com/@{author_uniqueid}/video/{video_id}"
    querystring = {"url": tiktok_url}
    headers = {
        "x-rapidapi-key": TIKTOK_API_KEY,
        "x-rapidapi-host": TIKTOK_API_HOST
    }

    try:
        response = requests.get(TIKTOK_API_URL, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        play_url = data.get("play")

        if play_url:
            # Update Airtable record with video download URL
            update_data = {
                "fields": {
                    "video_download": play_url
                }
            }
            update_response = requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
            update_response.raise_for_status()
            print(f"Updated record {record_id} with video download URL.")
        else:
            print(f"No playable video found for record {record_id}.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch video for record {record_id}: {e}")


KeyboardInterrupt: 